In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pandas as pd
import numpy as np
import glob
import os
import json
from tqdm import tqdm

# ========== Paths (fill yours) ==========
BASE_FILE = "/content/drive/Othercomputers/我的 Mac/pm_stats/base_stations.csv"
DATA_FOLDER = "/content/drive/Othercomputers/我的 Mac/pm_stats/2024S-Dot"
OUTPUT_JSON = "/content/drive/Othercomputers/我的 Mac/pm_stats/results/summary.json"

# ========== Controls ==========
DATE_START = "2024-01-01"
DATE_END   = "2024-01-31"

# Start AFTER this serial in base (None = process all)
START_AFTER_SERIAL = None  # e.g. "OC3CL200102" or None

# Select metrics to process: "ALL" or comma-separated keys, e.g. "TEMP,NO2,CO"
SELECTED_METRICS_STR = "TEMP"  # change as needed

# ========== Metric dictionary (key -> (max_col, mean_col, min_col)) ==========
METRIC_MAP = {
    # Pollutants
    "NO2": ("이산화질소 최대(ppm)", "이산화질소 평균(ppm)", "이산화질소 최소(ppm)"),
    "CO":  ("일산화탄소 최대(ppm)", "일산화탄소 평균(ppm)", "일산화탄소 최소(ppm)"),
    "SO2": ("이산화황 최대(ppm)", "이산화황 평균(ppm)", "이산화황 최소(ppm)"),
    "NH3": ("암모니아 최대(ppm)", "암모니아 평균(ppm)", "암모니아 최소(ppm)"),
    "H2S": ("황화수소 최대(ppm)", "황화수소 평균(ppm)", "황화수소 최소(ppm)"),
    "O3":  ("오존 최대(ppm)",     "오존 평균(ppm)",     "오존 최소(ppm)"),
    # (Optional) environmental metrics, keep commented unless explicitly needed
     "TEMP": ("온도 최대(℃)", "온도 평균(℃)", "온도 최소(℃)"),
     "HUM":  ("습도 최대(%)", "습도 평균(%)", "습도 최소(%)"),
}

# Wind columns to attach at the timestamps of max/min if present
WIND_COLS = ["풍속 최대(m/s)", "풍속 평균(m/s)", "풍속 최소(m/s)",
             "풍향 최대(m/s)", "풍향 평균(m/s)", "풍향 최소(m/s)"]

In [20]:
# ========== Helpers ==========
def clean_header(df):
    """去掉表头空格并处理重复列名"""
    cols = df.columns.str.strip()
    seen = {}
    new_cols = []
    for col in cols:
        if col in seen:
            seen[col] += 1
            new_cols.append(f"{col}.{seen[col]}")
        else:
            seen[col] = 0
            new_cols.append(col)
    df.columns = new_cols
    return df

def read_csv_safe(path: str) -> pd.DataFrame:
    """Robust CSV reader handling utf-8-sig/cp949 and bad lines; cleans header."""
    try:
        df = pd.read_csv(path, encoding="utf-8-sig", low_memory=False, index_col=False, on_bad_lines="skip")
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding="cp949", low_memory=False, index_col=False, on_bad_lines="skip")
    return clean_header(df)

def resolve_col(cols_present, base_name: str):
    """Resolve a column by exact match; if not found, try startswith (handles '.1' duplicates)."""
    if base_name in cols_present:
        return base_name
    cands = [c for c in cols_present if c.startswith(base_name)]
    return cands[0] if cands else None

def parse_selected(selected_str: str):
    if selected_str.strip().upper() == "ALL":
        return list(METRIC_MAP.keys())
    items = [s.strip().upper() for s in selected_str.split(",") if s.strip()]
    return [k for k in items if k in METRIC_MAP]

# ========== Load base and station list ==========
base_df = pd.read_csv(BASE_FILE, encoding="utf-8-sig")
start_idx = 0
if START_AFTER_SERIAL is not None and START_AFTER_SERIAL in base_df["시리얼"].values:
    start_idx = base_df.index[base_df["시리얼"] == START_AFTER_SERIAL][0] + 1
work_base = base_df.iloc[start_idx:].copy()

# Station meta and set
stations = []
station_info_map = {}
for _, r in work_base.iterrows():
    serial = r["시리얼"]
    stations.append(serial)
    station_info_map[serial] = {
        "모델번호": r.get("모델번호"),
        "시리얼": r.get("시리얼"),
        "지역": r.get("지역"),
        "자치구": r.get("자치구"),
        "행정동": r.get("행정동"),
    }
station_set = set(stations)

# Date range scaffold
date_index = pd.date_range(DATE_START, DATE_END, freq="D")
date_strs = [d.strftime("%Y-%m-%d") for d in date_index]

# Which metrics
selected_keys = parse_selected(SELECTED_METRICS_STR)
if not selected_keys:
    raise ValueError("No valid metrics selected. Check SELECTED_METRICS_STR and METRIC_MAP keys.")
metrics_to_process = {k: METRIC_MAP[k] for k in selected_keys}

# ========== Aggregation cache (incremental) ==========
# agg[serial][date][metric] = {
#   'count': int,
#   'sum_mean': float,
#   'n_mean': int,
#   'max_val': float or None,
#   'max_time': str or None,
#   'max_winds': {wc: val or None},
#   'min_val': float or None,
#   'min_time': str or None,
#   'min_winds': {wc: val or None}
# }
agg = {s: {} for s in stations}

def ensure_day_metric(serial, day_str, metric):
    if day_str not in agg[serial]:
        agg[serial][day_str] = {}
    if metric not in agg[serial][day_str]:
        agg[serial][day_str][metric] = {
            'count': 0,
            'sum_mean': 0.0,
            'n_mean': 0,
            'max_val': None, 'max_time': None, 'max_winds': {},
            'min_val': None, 'min_time': None, 'min_winds': {},
        }

# ========== Pre-init final_result with station_info and empty 'data' ==========
final_result = {s: {"station_info": station_info_map[s], "data": {}} for s in stations}

# ========== Iterate files (outer), update agg (inner) ==========
weekly_files = sorted(glob.glob(os.path.join(DATA_FOLDER, "*.csv")))
if not weekly_files:
    raise FileNotFoundError("No weekly CSV files found in DATA_FOLDER.")

for fp in tqdm(weekly_files, desc="Files"):
    dfw = read_csv_safe(fp)
    if "시리얼" not in dfw.columns or "측정시간" not in dfw.columns:
        continue

    # Keep only stations we care about
    dfw = dfw[dfw["시리얼"].isin(station_set)]
    if dfw.empty:
        continue

    # Build date column (YYYY-MM-DD)
    dfw["__date__"] = pd.to_datetime(dfw["측정시간"].astype(str).str[:10], errors="coerce")
    dfw = dfw[dfw["__date__"].notna()]
    dfw = dfw[(dfw["__date__"] >= pd.to_datetime(DATE_START)) & (dfw["__date__"] <= pd.to_datetime(DATE_END))]
    if dfw.empty:
        continue

    # For each metric, compute group aggregates on the fly and update agg
    for key, (max_col, mean_col, min_col) in metrics_to_process.items():
        cols_present = dfw.columns
        r_max  = resolve_col(cols_present, max_col)
        r_mean = resolve_col(cols_present, mean_col)
        r_min  = resolve_col(cols_present, min_col)

        # If metric columns are completely missing in this file, skip
        if r_max is None and r_mean is None and r_min is None:
            continue

        s_max  = pd.to_numeric(dfw[r_max],  errors="coerce") if r_max  else pd.Series([pd.NA]*len(dfw), index=dfw.index)
        s_mean = pd.to_numeric(dfw[r_mean], errors="coerce") if r_mean else pd.Series([pd.NA]*len(dfw), index=dfw.index)
        s_min  = pd.to_numeric(dfw[r_min],  errors="coerce") if r_min  else pd.Series([pd.NA]*len(dfw), index=dfw.index)

        # Count = number of rows where ANY of the 3 are present
        any_vals = (s_max.notna() | s_mean.notna() | s_min.notna())

        # --- Group-level sums ---
        grp = dfw.groupby(["시리얼", "__date__"], sort=False)

        # counts
        count_series = any_vals.groupby([dfw["시리얼"], dfw["__date__"]]).sum()

        # mean sum and n
        mean_sum = s_mean.where(s_mean.notna(), 0.0).groupby([dfw["시리얼"], dfw["__date__"]]).sum()
        mean_n   = s_mean.notna().groupby([dfw["시리얼"], dfw["__date__"]]).sum()

        # max/min values
        max_val = s_max.groupby([dfw["시리얼"], dfw["__date__"]]).max(min_count=1)
        min_val = s_min.groupby([dfw["시리얼"], dfw["__date__"]]).min()

        # --- First-occurrence rows of group max/min (robust across pandas versions) ---
        # Build helper frame with needed cols
        cols_for_pick = ["시리얼", "__date__", "측정시간"] + [wc for wc in WIND_COLS if wc in dfw.columns]
        pick = dfw[cols_for_pick].copy()

        # Group keys
        gkeys = [dfw["시리얼"], dfw["__date__"]]

        # idx of first non-null max/min per group (NaN if group is all NaN)
        idx_max = s_max.groupby(gkeys).idxmax()
        idx_min = s_min.groupby(gkeys).idxmin()

        # Drop groups with no valid extrema
        idx_max = idx_max.dropna().astype(int)
        idx_min = idx_min.dropna().astype(int)

        # Lookups for (serial, day_str) -> value / time / winds
        max_val_dict = {}
        min_val_dict = {}
        max_row_lookup = {}
        min_row_lookup = {}

        # Fill max dicts
        for (serial_k, day_k), i in idx_max.items():
            val = s_max.iloc[i]
            day_str = pd.to_datetime(day_k).strftime("%Y-%m-%d")
            max_val_dict[(serial_k, day_str)] = float(val) if pd.notna(val) else None

            row = pick.iloc[i]
            info = {"time": row["측정시간"]}
            for wc in WIND_COLS:
                if wc in pick.columns:
                    info[wc] = row.get(wc, None)
            max_row_lookup[(serial_k, day_str)] = info

        # Fill min dicts
        for (serial_k, day_k), i in idx_min.items():
            val = s_min.iloc[i]
            day_str = pd.to_datetime(day_k).strftime("%Y-%m-%d")
            min_val_dict[(serial_k, day_str)] = float(val) if pd.notna(val) else None

            row = pick.iloc[i]
            info = {"time": row["측정시간"]}
            for wc in WIND_COLS:
                if wc in pick.columns:
                    info[wc] = row.get(wc, None)
            min_row_lookup[(serial_k, day_str)] = info



        # --- Update agg cache ---
        for (serial, day), c in count_series.items():
            day_str = pd.to_datetime(day).strftime("%Y-%m-%d")
            if serial not in station_set:
                continue
            if day_str not in date_strs:
                continue
            ensure_day_metric(serial, day_str, key)

            agg_item = agg[serial][day_str][key]
            agg_item['count'] += int(c)

        for (serial, day), s in mean_sum.items():
            day_str = pd.to_datetime(day).strftime("%Y-%m-%d")
            if serial not in station_set or day_str not in date_strs:
                continue
            ensure_day_metric(serial, day_str, key)
            agg[serial][day_str][key]['sum_mean'] += float(s)

        for (serial, day), n in mean_n.items():
            day_str = pd.to_datetime(day).strftime("%Y-%m-%d")
            if serial not in station_set or day_str not in date_strs:
                continue
            ensure_day_metric(serial, day_str, key)
            agg[serial][day_str][key]['n_mean'] += int(n)

        for (serial, day_str), v in max_val_dict.items():
            if serial not in station_set or day_str not in date_strs:
                continue
            ensure_day_metric(serial, day_str, key)
            cur = agg[serial][day_str][key]
            # Update only if new max is greater
            if v is not None and (cur['max_val'] is None or v > cur['max_val']):
                cur['max_val'] = v
                info = max_row_lookup.get((serial, day_str), {})
                cur['max_time'] = info.get("time")
                cur['max_winds'] = {wc: info.get(wc) for wc in WIND_COLS}

        for (serial, day_str), v in min_val_dict.items():
            if serial not in station_set or day_str not in date_strs:
                continue
            ensure_day_metric(serial, day_str, key)
            cur = agg[serial][day_str][key]
            # Update only if new min is smaller
            if v is not None and (cur['min_val'] is None or v < cur['min_val']):
                cur['min_val'] = v
                info = min_row_lookup.get((serial, day_str), {})
                cur['min_time'] = info.get("time")
                cur['min_winds'] = {wc: info.get(wc) for wc in WIND_COLS}

# ========== Finalize: build JSON from agg ==========
for serial in stations:
    # Ensure all dates exist in output, even if never observed (count=0, others None)
    for day_str in date_strs:
        day_out = {}
        for key in metrics_to_process.keys():
            it = agg.get(serial, {}).get(day_str, {}).get(key, None)
            if it is None:
                # No observation at all for this metric/date
                day_out[f"{key}_count"] = 0
                day_out[f"{key}_max"] = None
                day_out[f"{key}_max_time"] = None
                day_out[f"{key}_min"] = None
                day_out[f"{key}_min_time"] = None
                day_out[f"{key}_mean"] = None
                for wc in WIND_COLS:
                    day_out[f"{key}_max_{wc}"] = None
                    day_out[f"{key}_min_{wc}"] = None
            else:
                # Mean from accumulated sum/count
                if it['n_mean'] > 0:
                    mean_val = it['sum_mean'] / it['n_mean']
                else:
                    mean_val = None
                day_out[f"{key}_count"] = int(it['count'])
                day_out[f"{key}_max"] = None if it['max_val'] is None else float(it['max_val'])
                day_out[f"{key}_max_time"] = it['max_time']
                day_out[f"{key}_min"] = None if it['min_val'] is None else float(it['min_val'])
                day_out[f"{key}_min_time"] = it['min_time']
                day_out[f"{key}_mean"] = None if mean_val is None else float(mean_val)
                # Wind snapshots at extrema
                for wc in WIND_COLS:
                    day_out[f"{key}_max_{wc}"] = it['max_winds'].get(wc) if it['max_winds'] else None
                    day_out[f"{key}_min_{wc}"] = it['min_winds'].get(wc) if it['min_winds'] else None

        final_result[serial]["data"][day_str] = day_out

Files:   2%|▏         | 1/53 [00:07<06:28,  7.48s/it]


KeyboardInterrupt: 

In [ ]:
# ========== Save JSON ==========
os.makedirs(os.path.dirname(OUTPUT_JSON), exist_ok=True)
with open(OUTPUT_JSON, "w", encoding="utf-8-sig") as f:
    json.dump(final_result, f, ensure_ascii=False, indent=2)

print(f"✅ JSON saved to {OUTPUT_JSON}")

In [7]:
import json
from datetime import datetime, timedelta
import numpy as np

# ===== Parameters =====
INPUT_JSON = "/content/drive/Othercomputers/我的 Mac/pm_stats/results/summary.json"
DATE_START = "2024-01-01"
DATE_END   = "2024-01-31"
SELECTED_METRIC = "TEMP"   # e.g., "TEMP", "NO2", "CO", "SO2", "NH3", "H2S", "O3"

# ===== Read JSON =====
with open(INPUT_JSON, "r", encoding="utf-8-sig") as f:
    data = json.load(f)

# Normalize metric key (JSON keys are uppercase prefixes like NO2, TEMP)
SELECTED_METRIC = SELECTED_METRIC.strip().upper()
count_key = f"{SELECTED_METRIC}_count"

# ===== Generate complete date list =====
start_dt = datetime.strptime(DATE_START, "%Y-%m-%d")
end_dt = datetime.strptime(DATE_END, "%Y-%m-%d")
all_days = [(start_dt + timedelta(days=i)).strftime("%Y-%m-%d")
            for i in range((end_dt - start_dt).days + 1)]
total_days = len(all_days)

# ===== Calculate coverage rate (per station) =====
coverage_list = []  # [(serial, valid_days, coverage_rate), ...]
metric_seen_somewhere = False

for serial, payload in data.items():
    day_map = payload.get("data", {})
    valid_days = 0
    for d in all_days:
        rec = day_map.get(d)
        if not rec:
            continue
        # Count this day only if SELECTED_METRIC has count > 0
        if count_key in rec:
            metric_seen_somewhere = True
            if (rec.get(count_key) or 0) > 0:
                valid_days += 1
        else:
            # If the metric-specific key is missing entirely on this day, treat as no data
            pass
    rate = valid_days / total_days if total_days else 0.0
    coverage_list.append((serial, valid_days, rate))

# ===== Output results =====
rates = [r for _, _, r in coverage_list]
print(f"[Metric] {SELECTED_METRIC}")
print(f"Total stations: {len(coverage_list)}")
print(f"Total days: {total_days}")
print(f"Average coverage rate: {np.mean(rates)*100:.2f}%")
print(f"Median coverage rate: {np.median(rates)*100:.2f}%")
print(f"Lowest coverage rate: {min(rates)*100:.2f}%")
print(f"Highest coverage rate: {max(rates)*100:.2f}%")

print("\nExample (first 10 stations):")
for serial, valid, rate in coverage_list[:10]:
    print(f"{serial}: {valid}/{total_days} days, coverage rate {rate*100:.2f}%")

if not metric_seen_somewhere:
    print(f"\n⚠️ The metric '{SELECTED_METRIC}' was not found in any day's record "
          f"(no '{count_key}' keys). Double-check your metric name and JSON content.")

[Metric] TEMP
Total stations: 1130
Total days: 31
Average coverage rate: 83.21%
Median coverage rate: 96.77%
Lowest coverage rate: 0.00%
Highest coverage rate: 96.77%

Example (first 10 stations):
OC3CL200011: 0/31 days, coverage rate 0.00%
OC3CL200020: 30/31 days, coverage rate 96.77%
OC3CL200026: 30/31 days, coverage rate 96.77%
OC3CL200027: 30/31 days, coverage rate 96.77%
OC3CL200012: 30/31 days, coverage rate 96.77%
OC3CL200017: 0/31 days, coverage rate 0.00%
OC3CL200032: 30/31 days, coverage rate 96.77%
OC3CL200030: 30/31 days, coverage rate 96.77%
OC3CL200022: 30/31 days, coverage rate 96.77%
OC3CL200025: 30/31 days, coverage rate 96.77%
